# Elliptical, parabolic, and hyperbolic orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  

In this notebook I plan to implement hyperbolic, elliptical, and parabolic orbits using the Euler method. Parabolic orbits are marginally bound, hyperbolic orbits are unbound, and elliptical orbits are bound but oblong. All orbits should have the center of mass at one focus for both stars orbit and be symmetric between the two stars. 

Elliptical orbits should be elongated away from the other star. 

Hyperbolic and parabolic orbits should open away from the other star. 

All orbits should conserve energy and angular momentum. Later this could be enforced. I will probably not do that in this notebook. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [2]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [3]:
import math
def InitialDataEqualMassConic(ecc,angle,initmass):
    orbitalangle=angle
    phi=np.array([orbitalangle,math.pi+orbitalangle])
    orbitalradius=100*np.ones(2) #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [4]:
import random,numpy as np
initdateqellipse=InitialDataEqualMassConic(0.1,math.pi/6,1.)
print(initdateqellipse)

0.1
(array([0.52359878, 3.66519143]), 0.5235987755982988, array([100., 100.]), 0.1, array([1., 1.]))


In [74]:
def getxyuveqconic(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    if eccentricity<1 and eccentricity>0:
        #elliptical
        E=-Gconstant*masses[0]*masses[1]/2/orbitalradius[0] #1/2*mv^2-GMm/r #orbitalradius is correct here r1+r2
        coordsep=orbitalradius[0]*(1.+eccentricity) #starts at apastron
        r0=2.*coordsep
        x0=coordsep*cosphi
        y0=coordsep*sinphi
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]*(2./np.abs(coordsep)-2.*E/masses[i]))
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        for i in np.arange(2):
            a[i]=Gconstant*masses[(i+1)%2]/(r0**2)
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        for i in np.arange(2):
            a[i]=Gconstant*masses[(i+1)%2]/(r0[i]**2)
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [75]:
xyuvaeqconic=getxyuveqconic(initdateqellipse)
print(xyuvaeqconic)

initdat [100. 100.] [0.52359878 3.66519143] [ 0.8660254 -0.8660254] [ 0.5 -0.5]
coordsep [50. 50.]
[ 43.30127019 -43.30127019]
[ 25. -25.]
0.1
(array([1., 1.]), array([ 95.26279442, -95.26279442]), array([ 55., -55.]), array([0., 0.]), array([-0.08393721,  0.08393721]), array([ 0.14538351, -0.14538351]), array([0., 0.]), array([-1.78930869e-05,  1.78930869e-05]), array([-1.03305785e-05,  1.03305785e-05]), array([0., 0.]))


In [76]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                #print(rreljk)
                axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        
        #print(vyii[m],vyi[m],ayi[m],dt)
        #print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [77]:
dt=1 #*31556926 #seconds per year
numsteps=10000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqconic
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
t=0.0
masstotal=1.
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[ 95.26279442 -95.26279442]
[ 95.17885721 -95.17885721] [ 55.14538351 -55.14538351] [-0.0839551  0.0839551] [ 0.14537317 -0.14537317] [-1.78930869e-05  1.78930869e-05] [-1.03305785e-05  1.03305785e-05]
[ 95.09490211 -95.09490211] [ 55.29075668 -55.29075668] [-0.08397299  0.08397299] [ 0.14536284 -0.14536284] [-1.78772586e-05  1.78772586e-05] [-1.03578495e-05  1.03578495e-05]
[ 95.01092912 -95.01092912] [ 55.43611952 -55.43611952] [-0.08399087  0.08399087] [ 0.14535249 -0.14535249] [-1.78613123e-05  1.78613123e-05] [-1.03850517e-05  1.03850517e-05]
[ 94.92693825 -94.92693825] [ 55.58147201 -55.58147201] [-0.08400873  0.08400873] [ 0.1453421 -0.1453421] [-1.78452484e-05  1.78452484e-05] [-1.04121845e-05  1.04121845e-05]
[ 94.84292952 -94.84292952] [ 55.72681411 -55.72681411] [-0.08402658  0.08402658] [ 0.14533169 -0.14533169] [-1.78290671e-05  1.78290671e-05] [-1.04392474e-05  1.04392474e-05]
[ 94.75890294 -94.75890294] [ 55.8721458 -55.8721458] [-0.08404441  0.08404441] [ 0.14532125 -0.

[ 78.1462088 -78.1462088] [ 83.84127763 -83.84127763] [-0.08707875  0.08707875] [ 0.14289524 -0.14289524] [-1.30058805e-05  1.30058805e-05] [-1.39144243e-05  1.39144243e-05]
[ 78.05913005 -78.05913005] [ 83.98417286 -83.98417286] [-0.08709176  0.08709176] [ 0.14288132 -0.14288132] [-1.29760886e-05  1.29760886e-05] [-1.39217482e-05  1.39217482e-05]
[ 77.97203829 -77.97203829] [ 84.12705418 -84.12705418] [-0.08710473  0.08710473] [ 0.1428674 -0.1428674] [-1.29462823e-05  1.29462823e-05] [-1.3928964e-05  1.3928964e-05]
[ 77.88493356 -77.88493356] [ 84.26992158 -84.26992158] [-0.08711768  0.08711768] [ 0.14285347 -0.14285347] [-1.29164624e-05  1.29164624e-05] [-1.39360719e-05  1.39360719e-05]
[ 77.79781588 -77.79781588] [ 84.41277505 -84.41277505] [-0.0871306  0.0871306] [ 0.14283953 -0.14283953] [-1.28866293e-05  1.28866293e-05] [-1.3943072e-05  1.3943072e-05]
[ 77.71068528 -77.71068528] [ 84.55561458 -84.55561458] [-0.08714348  0.08714348] [ 0.14282559 -0.14282559] [-1.28567835e-05  1.28

[ 63.53527924 -63.53527924] [ 107.36892915 -107.36892915] [-0.08883351  0.08883351] [ 0.14055915 -0.14055915] [-8.2060832e-06  8.2060832e-06] [-1.38300542e-05  1.38300542e-05]
[ 63.44644573 -63.44644573] [ 107.5094883 -107.5094883] [-0.08884172  0.08884172] [ 0.14054532 -0.14054532] [-8.17972784e-06  8.17972784e-06] [-1.38230072e-05  1.38230072e-05]
[ 63.35760402 -63.35760402] [ 107.65003361 -107.65003361] [-0.0888499  0.0888499] [ 0.14053149 -0.14053149] [-8.15341787e-06  8.15341787e-06] [-1.38159005e-05  1.38159005e-05]
[ 63.26875412 -63.26875412] [ 107.79056511 -107.79056511] [-0.08885805  0.08885805] [ 0.14051768 -0.14051768] [-8.12715346e-06  8.12715346e-06] [-1.38087347e-05  1.38087347e-05]
[ 63.17989607 -63.17989607] [ 107.93108279 -107.93108279] [-0.08886618  0.08886618] [ 0.14050387 -0.14050387] [-8.10093475e-06  8.10093475e-06] [-1.38015099e-05  1.38015099e-05]
[ 63.09102989 -63.09102989] [ 108.07158666 -108.07158666] [-0.08887428  0.08887428] [ 0.14049007 -0.14049007] [-8.07

[ 48.60489589 -48.60489589] [ 130.65750888 -130.65750888] [-0.08987648  0.08987648] [ 0.1383825 -0.1383825] [-4.50302317e-06  4.50302317e-06] [-1.20696868e-05  1.20696868e-05]
[ 48.51501941 -48.51501941] [ 130.79589138 -130.79589138] [-0.08988098  0.08988098] [ 0.13837043 -0.13837043] [-4.48520961e-06  4.48520961e-06] [-1.205694e-05  1.205694e-05]
[ 48.42513842 -48.42513842] [ 130.93426181 -130.93426181] [-0.08988547  0.08988547] [ 0.13835837 -0.13835837] [-4.46744664e-06  4.46744664e-06] [-1.20441808e-05  1.20441808e-05]
[ 48.33525296 -48.33525296] [ 131.07262018 -131.07262018] [-0.08988993  0.08988993] [ 0.13834633 -0.13834633] [-4.44973419e-06  4.44973419e-06] [-1.20314093e-05  1.20314093e-05]
[ 48.24536302 -48.24536302] [ 131.21096651 -131.21096651] [-0.08989438  0.08989438] [ 0.1383343 -0.1383343] [-4.43207221e-06  4.43207221e-06] [-1.20186258e-05  1.20186258e-05]
[ 48.15546864 -48.15546864] [ 131.3493008 -131.3493008] [-0.08989882  0.08989882] [ 0.13832228 -0.13832228] [-4.414460

[ 34.4492391 -34.4492391] [ 152.24399219 -152.24399219] [-0.09039544  0.09039544] [ 0.13664734 -0.13664734] [-2.2753831e-06  2.2753831e-06] [-1.00204442e-05  1.00204442e-05]
[ 34.35884366 -34.35884366] [ 152.38063953 -152.38063953] [-0.09039772  0.09039772] [ 0.13663732 -0.13663732] [-2.26449031e-06  2.26449031e-06] [-1.00076244e-05  1.00076244e-05]
[ 34.26844595 -34.26844595] [ 152.51727685 -152.51727685] [-0.09039998  0.09039998] [ 0.13662731 -0.13662731] [-2.25363413e-06  2.25363413e-06] [-9.99481279e-06  9.99481279e-06]
[ 34.17804597 -34.17804597] [ 152.65390416 -152.65390416] [-0.09040223  0.09040223] [ 0.13661732 -0.13661732] [-2.24281445e-06  2.24281445e-06] [-9.9820095e-06  9.9820095e-06]
[ 34.08764373 -34.08764373] [ 152.79052147 -152.79052147] [-0.09040448  0.09040448] [ 0.13660733 -0.13660733] [-2.23203118e-06  2.23203118e-06] [-9.96921459e-06  9.96921459e-06]
[ 33.99723926 -33.99723926] [ 152.92712881 -152.92712881] [-0.09040671  0.09040671] [ 0.13659736 -0.13659736] [-2.22

...

[-734.80706271  734.80706271] [ 1256.20868716 -1256.20868716] [-0.08888078  0.08888078] [ 0.12681132 -0.12681132] [ 5.96086822e-08 -5.96086822e-08] [-1.01907626e-07  1.01907626e-07]
[-734.89594349  734.89594349] [ 1256.33549848 -1256.33549848] [-0.08888072  0.08888072] [ 0.12681122 -0.12681122] [ 5.95969286e-08 -5.95969286e-08] [-1.01885492e-07  1.01885492e-07]
[-734.98482421  734.98482421] [ 1256.46230971 -1256.46230971] [-0.08888066  0.08888066] [ 0.12681112 -0.12681112] [ 5.95851785e-08 -5.95851785e-08] [-1.01863367e-07  1.01863367e-07]
[-735.07370487  735.07370487] [ 1256.58912083 -1256.58912083] [-0.0888806  0.0888806] [ 0.12681102 -0.12681102] [ 5.95734318e-08 -5.95734318e-08] [-1.01841248e-07  1.01841248e-07]
[-735.16258547  735.16258547] [ 1256.71593184 -1256.71593184] [-0.08888054  0.08888054] [ 0.12681092 -0.12681092] [ 5.95616886e-08 -5.95616886e-08] [-1.01819137e-07  1.01819137e-07]
[-735.25146601  735.25146601] [ 1256.84274276 -1256.84274276] [-0.08888048  0.08888048] [ 0.

[-749.1161081  749.1161081] [ 1276.62401255 -1276.62401255] [-0.08887133  0.08887133] [ 0.1267952 -0.1267952] [ 5.77599242e-08 -5.77599242e-08] [-9.84348484e-08  9.84348484e-08]
[-749.20497943  749.20497943] [ 1276.75080775 -1276.75080775] [-0.08887127  0.08887127] [ 0.1267951 -0.1267951] [ 5.77487096e-08 -5.77487096e-08] [-9.84138354e-08  9.84138354e-08]
[-749.2938507  749.2938507] [ 1276.87760285 -1276.87760285] [-0.08887121  0.08887121] [ 0.126795 -0.126795] [ 5.77374982e-08 -5.77374982e-08] [-9.83928291e-08  9.83928291e-08]
[-749.38272192  749.38272192] [ 1277.00439785 -1277.00439785] [-0.08887116  0.08887116] [ 0.1267949 -0.1267949] [ 5.77262901e-08 -5.77262901e-08] [-9.83718295e-08  9.83718295e-08]
[-749.47159308  749.47159308] [ 1277.13119275 -1277.13119275] [-0.0888711  0.0888711] [ 0.1267948 -0.1267948] [ 5.77150853e-08 -5.77150853e-08] [-9.83508367e-08  9.83508367e-08]
[-749.56046417  749.56046417] [ 1277.25798755 -1277.25798755] [-0.08887104  0.08887104] [ 0.1267947 -0.12679

[-763.95682802  763.95682802] [ 1297.79746175 -1297.79746175] [-0.08886184  0.08886184] [ 0.12677904 -0.12677904] [ 5.59311313e-08 -5.59311313e-08] [-9.50166685e-08  9.50166685e-08]
[-764.04568986  764.04568986] [ 1297.92424079 -1297.92424079] [-0.08886178  0.08886178] [ 0.12677895 -0.12677895] [ 5.59204418e-08 -5.59204418e-08] [-9.4996739e-08  9.4996739e-08]
[-764.13455164  764.13455164] [ 1298.05101974 -1298.05101974] [-0.08886173  0.08886173] [ 0.12677885 -0.12677885] [ 5.59097553e-08 -5.59097553e-08] [-9.49768157e-08  9.49768157e-08]
[-764.22341336  764.22341336] [ 1298.1777986 -1298.1777986] [-0.08886167  0.08886167] [ 0.12677876 -0.12677876] [ 5.58990719e-08 -5.58990719e-08] [-9.49568987e-08  9.49568987e-08]
[-764.31227503  764.31227503] [ 1298.30457736 -1298.30457736] [-0.08886161  0.08886161] [ 0.12677866 -0.12677866] [ 5.58883915e-08 -5.58883915e-08] [-9.4936988e-08  9.4936988e-08]
[-764.40113664  764.40113664] [ 1298.43135602 -1298.43135602] [-0.08886156  0.08886156] [ 0.1267

[-778.70713488  778.70713488] [ 1318.84149653 -1318.84149653] [-0.0888527  0.0888527] [ 0.12676354 -0.12676354] [ 5.41975315e-08 -5.41975315e-08] [-9.17921977e-08  9.17921977e-08]
[-778.79598758  778.79598758] [ 1318.96826007 -1318.96826007] [-0.08885264  0.08885264] [ 0.12676345 -0.12676345] [ 5.41873321e-08 -5.41873321e-08] [-9.17732727e-08  9.17732727e-08]
[-778.88484022  778.88484022] [ 1319.09502352 -1319.09502352] [-0.08885259  0.08885259] [ 0.12676336 -0.12676336] [ 5.41771357e-08 -5.41771357e-08] [-9.17543535e-08  9.17543535e-08]
[-778.97369281  778.97369281] [ 1319.22178687 -1319.22178687] [-0.08885253  0.08885253] [ 0.12676326 -0.12676326] [ 5.4166942e-08 -5.4166942e-08] [-9.17354402e-08  9.17354402e-08]
[-779.06254534  779.06254534] [ 1319.34855014 -1319.34855014] [-0.08885248  0.08885248] [ 0.12676317 -0.12676317] [ 5.41567513e-08 -5.41567513e-08] [-9.17165327e-08  9.17165327e-08]
[-779.15139782  779.15139782] [ 1319.47531331 -1319.47531331] [-0.08885243  0.08885243] [ 0.12

[-790.16868881  790.16868881] [ 1335.19324184 -1335.19324184] [-0.08884579  0.08884579] [ 0.12675185 -0.12675185] [ 5.29050776e-08 -5.29050776e-08] [-8.93983018e-08  8.93983018e-08]
[-790.25753459  790.25753459] [ 1335.31999369 -1335.31999369] [-0.08884574  0.08884574] [ 0.12675176 -0.12675176] [ 5.28952388e-08 -5.28952388e-08] [-8.93801114e-08  8.93801114e-08]
[-790.34638033  790.34638033] [ 1335.44674546 -1335.44674546] [-0.08884568  0.08884568] [ 0.12675167 -0.12675167] [ 5.28854028e-08 -5.28854028e-08] [-8.93619265e-08  8.93619265e-08]
[-790.43522601  790.43522601] [ 1335.57349713 -1335.57349713] [-0.08884563  0.08884563] [ 0.12675158 -0.12675158] [ 5.28755695e-08 -5.28755695e-08] [-8.93437472e-08  8.93437472e-08]
[-790.52407164  790.52407164] [ 1335.70024872 -1335.70024872] [-0.08884558  0.08884558] [ 0.1267515 -0.1267515] [ 5.28657389e-08 -5.28657389e-08] [-8.93255734e-08  8.93255734e-08]
[-790.61291722  790.61291722] [ 1335.82700021 -1335.82700021] [-0.08884552  0.08884552] [ 0.

In [78]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, Equal Mass System, Ellipse", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=1, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(xcoord2,ycoord2, size=1, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="mu")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

The center of mass isn't at the ellipse focus. The two ellipses should be alligned so that the axises are pointed in the same direction-- the long axises should be parallel and overlapping. But this is in fact two ellipses!